In [ ]:
# Импортируем необходимые библиотеки
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import nltk
from collections import Counter
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.cuda.amp import GradScaler, autocast

In [ ]:
# Установим случайные начальные значения для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Проверим доступность GPU видеокарты
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

In [ ]:
# 1. Предварительная обработка данных

class VideoDataset(Dataset):
    """Датасет для работы с видео и их описаниями"""
    
    def __init__(self, feature_dir, caption_file, vocab, max_frames=40):
        """Инициализация датасета
        
        Аргументы:
            feature_dir (str): Путь к директории с предвычисленными признаками
            caption_file (str): Файл с описаниями в формате "video_id описание"
            vocab (Vocabulary): Объект словаря для токенизации
            max_frames (int): Макс. количество кадров на видео
        """
        self.feature_dir = feature_dir
        self.max_frames = max_frames
        self.vocab = vocab
        
        # Загрузка и парсинг описаний
        self.captions = self._load_captions(caption_file)
        self.video_ids = list(self.captions.keys())
    
    def _load_captions(self, caption_file):
        """Загружает описания из файла"""
        captions = {}
        with open(caption_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(' ', 1)
                if len(parts) == 2:
                    video_id, caption = parts
                    captions.setdefault(video_id, []).append(caption)
        return captions
    
    def __len__(self):
        return len(self.video_ids)
    
    def __getitem__(self, idx):
        """Получает один элемент датасета по индексу"""
        video_id = self.video_ids[idx]
        
        # Загрузка предвычисленных признаков
        features = self._load_features(video_id)
        
        # Выбор случайного описания и токенизация
        caption = self._process_caption(video_id)
        
        return features, caption
    
    def _load_features(self, video_id):
        """Загружает признаки видео из файла"""
        feature_path = os.path.join(self.feature_dir, f"{video_id}.npy")
        features = np.load(feature_path)
        
        # Проверяем и корректируем размерность
        if features.ndim > 2:            
            features = features.reshape(features.shape[0], -1) # Преобразуем к [seq_len, feature_dim]
        return torch.FloatTensor(features).to(device)
    
    def _process_caption(self, video_id):
        """Токенизирует и преобразует описание в тензор"""
        caption = np.random.choice(self.captions[video_id])
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        caption = [self.vocab('<start>')] + [self.vocab(token) for token in tokens] + [self.vocab('<end>')]
        return torch.LongTensor(caption)

class Vocabulary:
    """Словарь для преобразования слов в индексы"""
    
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        self._add_special_tokens()
    
    def _add_special_tokens(self):
        """Добавляет специальные токены"""
        for token in ['<pad>', '<start>', '<end>', '<unk>']:
            self.add_word(token)
    
    def add_word(self, word):
        """Добавляет слово в словарь"""
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __call__(self, word):
        """Возвращает индекс слова или токен <unk>"""
        return self.word2idx.get(word, self.word2idx['<unk>'])
    
    def __len__(self):
        return len(self.word2idx)

def build_vocab(caption_file, threshold=4):
    """Строит словарь на основе файла с описаниями"""
    counter = Counter()
    
    # Подсчет частот слов
    with open(caption_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2:
                counter.update(nltk.tokenize.word_tokenize(parts[1].lower()))
    
    # Фильтрация по порогу
    vocab = Vocabulary()
    for word, count in counter.items():
        if count >= threshold:
            vocab.add_word(word)
    
    return vocab

def precompute_features(video_dir, output_dir, batch_size=16):
    """Предварительно вычисляет признаки видео с помощью ResNet152"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Инициализация модели для извлечения признаков
    model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
    feature_extractor = nn.Sequential(*list(model.children())[:-1]).to(device)
    feature_extractor.eval()
    
    # Трансформации для кадров
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Обработка видеофайлов
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]
    
    for i in range(0, len(video_files), batch_size):
        batch_files = video_files[i:i + batch_size]
        
        for video_file in tqdm(batch_files, desc=f"Batch {i//batch_size + 1}"):
            
            video_id = os.path.splitext(video_file)[0]
            feature_path = os.path.join(output_dir, f"{video_id}.npy") # Проверяем, существуют ли уже признаки
            if os.path.exists(feature_path):
                continue  # Признаки уже есть, пропускаем

            video_path = os.path.join(video_dir, video_file)
            # Извлечение кадров
            frames = extract_frames(video_path)
            
            # Извлечение признаков
            features = []
            with torch.no_grad():
                for frame in frames:
                    if frame.ndim == 3:  # Проверка валидности кадра
                        frame = transform(frame).unsqueeze(0).to(device)
                        feature = feature_extractor(frame).squeeze().cpu().numpy()
                        features.append(feature)
            
            # Сохранение признаков
            features = np.stack(features, axis=0)
            np.save(os.path.join(output_dir, f"{video_id}.npy"), features)

def extract_frames(video_path, max_frames=40):
    """Извлекает кадры из видео с равномерной дискретизацией"""
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(1, frame_count // max_frames)
    frames = []
    
    for i in range(0, frame_count, sample_rate):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if len(frames) >= max_frames:
            break
    
    cap.release()
    
    # Дополнение нулями при необходимости
    if len(frames) < max_frames:
        frames.extend([np.zeros_like(frames[0]) for _ in range(max_frames - len(frames))])
    
    return frames

In [ ]:
# 2. Извлечение признаков

class FeatureExtractor:
    """
    Класс для извлечения признаков из видеокадров с использованием предобученной CNN
    Основные функции:
    - Инициализация предобученной модели CNN (по умолчанию ResNet152)
    - Преобразование входных кадров к нужному формату
    - Извлечение признаков из каждого кадра
    """
    def __init__(self, cnn_model=None):
        if cnn_model is None:
            # Загрузка предобученной ResNet152
            cnn_model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
            # Удаляем последний классификационный слой
            self.model = nn.Sequential(*list(cnn_model.children())[:-1])
        else:
            self.model = cnn_model

        # Перенос модели на устройство (GPU/CPU) и перевод в режим оценки
        self.model = self.model.to(device)
        self.model.eval()
        
        # Определение преобразований для входных изображений
        self.transform = transforms.Compose([
            transforms.ToPILImage(),               # Конвертация в PIL Image
            transforms.Resize((224, 224)),         # Изменение размера под вход сети
            transforms.ToTensor(),                 # Конвертация в тензор
            transforms.Normalize(                  # Нормализация
                mean=[0.485, 0.456, 0.406],        # Средние значения ImageNet
                std=[0.229, 0.224, 0.225]          # Стандартные отклонения ImageNet
            )
        ])
    
    def extract_features(self, frames):
        """
        Извлекает признаки из списка кадров
        
        Аргументы:
            frames (list): Список кадров в формате numpy arrays
            
        Возвращает:
            torch.Tensor: Извлеченные признаки размерности [число_кадров, размерность_признака]
            
        Процесс работы:
        1. Применение преобразований к каждому кадру
        2. Извлечение признаков с помощью CNN
        3. Накопление и объединение признаков
        """
        features = []
        
        with torch.no_grad(): # Отключаем вычисление градиентов для ускорения
            for frame in frames:
                # Применяем преобразования и добавляем batch-размерность
                frame = self.transform(frame).unsqueeze(0).to(device)
                
                # Извлекаем признаки
                feature = self.model(frame)
                feature = feature.squeeze() # Удаляем лишние размерности
                
                features.append(feature.cpu()) # Переносим на CPU для экономии памяти
        
        return torch.stack(features) # Объединяем все признаки в один тензор

In [ ]:
# 3. Архитектура модели

class Encoder(nn.Module):
    """
    Видео-энкодер для обработки признаков кадров с учетом временной информации
    Использует двунаправленный LSTM для анализа последовательности кадров
    
    Основные функции:
    - Обработка признаков отдельных кадров
    - Учет временных зависимостей между кадрами
    - Подготовка скрытых состояний для декодера
    """
    def __init__(self, feature_dim, hidden_dim, num_layers=1, dropout=0.5):
        """
        Инициализация энкодера
        
        Аргументы:
            feature_dim (int): Размерность входных признаков кадра
            hidden_dim (int): Размерность скрытого слоя LSTM
            num_layers (int): Количество слоев LSTM
            dropout (float): Вероятность дропаута
        """
        super(Encoder, self).__init__()
        
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Двунаправленный LSTM для временного кодирования
        self.lstm = nn.LSTM(
            input_size=feature_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,          # Первая размерность - batch
            bidirectional=True,       # Двунаправленная архитектура
            dropout=dropout if num_layers > 1 else 0  # Дропаут только для многослойных LSTM
        )
        
    def forward(self, features):
        """
        Прямой проход через энкодер
        
        Аргументы:
            features (torch.Tensor): Признаки видеокадров размерности 
                                    [batch_size, seq_len, feature_dim]
            
        Возвращает:
            tuple: (outputs, hidden)
                - outputs: Выходы LSTM [batch_size, seq_len, hidden_dim*2]
                - hidden: Кортеж (скрытое состояние, состояние ячейки)
        """
         # Проверяем размерность входных данных
        if features.dim() > 3:
            batch_size, seq_len = features.size(0), features.size(1)
            # Преобразуем к [batch_size, seq_len, feature_dim]
            features = features.view(batch_size, seq_len, -1)
            
        # Убеждаемся, что последнее измерение имеет правильный размер
        if features.size(-1) != self.feature_dim:
            raise ValueError(f"Неверная размерность признаков: ожидается {self.feature_dim}, получено {features.size(-1)}")
        
        # Пропускаем признаки через LSTM
        outputs, hidden = self.lstm(features)
        
        return outputs, hidden

class AttentionLayer(nn.Module):
    """
    Слой внимания для выделения наиболее релевантных частей видео
    Реализует механизм внимания на основе скалярного произведения
    
    Основные функции:
    - Вычисление весов внимания для каждого кадра
    - Создание контекстного вектора
    """
    def __init__(self, encoder_dim, decoder_dim):
        """
        Инициализация слоя внимания
        
        Аргументы:
            encoder_dim (int): Размерность выхода энкодера
            decoder_dim (int): Размерность скрытого состояния декодера
        """
        super(AttentionLayer, self).__init__()

        # Линейные преобразования для вычисления внимания        
        self.encoder_attn = nn.Linear(encoder_dim, decoder_dim)
        self.full_attn = nn.Linear(decoder_dim, 1)
        
    def forward(self, encoder_outputs, decoder_hidden):
        """
        Прямой проход через слой внимания
        
        Аргументы:
            encoder_outputs (torch.Tensor): Выходы энкодера [batch_size, seq_len, encoder_dim]
            decoder_hidden (torch.Tensor): Скрытое состояние декодера [batch_size, decoder_dim]
            
        Возвращает:
            tuple: (context, attention_weights)
                - context: Контекстный вектор [batch_size, encoder_dim]
                - attention_weights: Веса внимания [batch_size, seq_len]
        """
        # Проецируем выходы энкодера в пространство декодера
        # Размерность: [batch_size, seq_len, decoder_dim]
        attn_proj = self.encoder_attn(encoder_outputs)
        
        # Добавляем размерность для совместимости
        # Размерность: [batch_size, 1, decoder_dim]
        decoder_hidden = decoder_hidden.unsqueeze(1)
        
        # Вычисляем оценки внимания через тангенс
        # Размерность: [batch_size, seq_len, 1]
        attn_scores = self.full_attn(torch.tanh(attn_proj + decoder_hidden))
        
        # Нормализуем оценки в веса с помощью softmax
        # Размерность: [batch_size, seq_len]
        attn_weights = F.softmax(attn_scores.squeeze(2), dim=1)
        
        # Вычисляем взвешенную сумму выходов энкодера
        # Размерность: [batch_size, encoder_dim]
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context, attn_weights

class Decoder(nn.Module):
    """
    Декодер подписей, генерирующий описание слово за словом
    Использует механизм внимания для фокусировки на релевантных частях видео
    
    Основные компоненты:
    - Слой эмбеддинга слов
    - Механизм внимания
    - LSTM ячейка
    - Выходной слой
    """
    def __init__(self, vocab_size, embed_dim, encoder_dim, hidden_dim, attention_dim, dropout=0.5):
        """
        Инициализация декодера
        
        Аргументы:
            vocab_size (int): Размер словаря
            embed_dim (int): Размерность эмбеддинга слов
            encoder_dim (int): Размерность выхода энкодера
            hidden_dim (int): Размерность скрытого слоя LSTM
            attention_dim (int): Размерность механизма внимания
            dropout (float): Вероятность дропаута
        """
        super(Decoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.encoder_dim = encoder_dim
        self.hidden_dim = hidden_dim
        
        # Слой эмбеддинга слов
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Слой внимания
        self.attention = AttentionLayer(encoder_dim, hidden_dim)
        
        # LSTM ячейка
        self.lstm = nn.LSTMCell(embed_dim + encoder_dim, hidden_dim)
        
        # Слой дропаута
        self.dropout = nn.Dropout(dropout)
        
        # Выходной полносвязный слой
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        # Инициализация весов
        self.init_weights()

    def init_weights(self):
        """Инициализация весов для эмбеддингов и выходного слоя."""
        self.embedding.weight.data.uniform_(-0.1, 0.1)  # Равномерная инициализация
        self.fc.bias.data.fill_(0)                      # Нулевые смещения
        self.fc.weight.data.uniform_(-0.1, 0.1)         # Равномерная инициализация
        
    def init_hidden_state(self, batch_size):
        """
        Инициализация скрытого состояния LSTM
        
        Аргументы:
            batch_size (int): Размер батча
            
        Возвращает:
            tuple: (скрытое состояние, состояние ячейки)
        """
        h = torch.zeros(batch_size, self.hidden_dim).to(device)
        c = torch.zeros(batch_size, self.hidden_dim).to(device)
        return h, c
        
    def forward(self, encoder_outputs, captions, lengths):
        """
        Прямой проход через декодер (обучение)
        
        Аргументы:
            encoder_outputs (torch.Tensor): Выходы энкодера [batch_size, seq_len, encoder_dim]
            captions (torch.Tensor): Истинные подписи [batch_size, max_caption_length]
            lengths (list): Фактические длины подписей
            
        Возвращает:
            torch.Tensor: Предсказания [batch_size, max_caption_length, vocab_size]
        """
        batch_size = encoder_outputs.size(0)
        
        # Сортируем данные по убыванию длины (для эффективности)
        lengths, sort_idx = torch.sort(lengths, descending=True)
        encoder_outputs = encoder_outputs[sort_idx]
        captions = captions[sort_idx]
        
        # Инициализируем состояние LSTM
        h, c = self.init_hidden_state(batch_size)
        
        # Определяем максимальную длину в батче
        max_length = lengths[0].item()
        
        # Инициализируем тензор предсказаний
        predictions = torch.zeros(batch_size, max_length, self.vocab_size).to(device)
        
        # Получаем эмбеддинги слов
        embeddings = self.embedding(captions)
        
        # Инициализируем контекстный вектор
        context, _ = self.attention(encoder_outputs, h)
        
        # Генерируем слова последовательно
        for t in range(max_length):
            # Объединяем эмбеддинг и контекстный вектор
            lstm_input = torch.cat([embeddings[:, t], context], dim=1)
            
            # Прямой проход через LSTM
            h, c = self.lstm(lstm_input, (h, c))
            
            # Вычисляем новый контекст
            context, _ = self.attention(encoder_outputs, h)
            
            # Генерируем предсказание следующего слова
            output = self.fc(self.dropout(h))
            predictions[:, t] = output
        
        return predictions
    
    def sample(self, encoder_outputs, max_length=20):
        """
        Генерация подписей для заданных выходов энкодера (инференс)
        
        Аргументы:
            encoder_outputs (torch.Tensor): Выходы энкодера [batch_size, seq_len, encoder_dim]
            max_length (int): Максимальная длина подписи
            
        Возвращает:
            list: Список сгенерированных подписей (индексы слов)
        """
        batch_size = encoder_outputs.size(0)
        
        # Инициализируем состояние LSTM
        h, c = self.init_hidden_state(batch_size)
        
        # Начинаем с токена <start>
        start_idx = torch.LongTensor([self.vocab.word2idx['<start>']]).to(device)
        embedding = self.embedding(start_idx)
        
        # Инициализируем контекстный вектор
        context, _ = self.attention(encoder_outputs, h)
        
        # Храним сгенерированные индексы
        sampled_ids = []
        
        # Генерируем слова последовательно
        for i in range(max_length):
            # Объединяем эмбеддинг и контекст
            lstm_input = torch.cat([embedding, context], dim=1)
            
            # Прямой проход через LSTM
            h, c = self.lstm(lstm_input, (h, c))
            
            # Вычисляем новый контекст
            context, alpha = self.attention(encoder_outputs, h)
            
            # Генерируем следующее слово
            output = self.fc(h)
            predicted = output.argmax(1)
            
            sampled_ids.append(predicted)
            
            # Прерываем если достигнут токен <end>
            if predicted.item() == self.vocab.word2idx['<end>']:
                break
            
            # Обновляем вход для следующего шага
            embedding = self.embedding(predicted)
        
        return torch.stack(sampled_ids, 1)

class VideoCaptioningModel(nn.Module):
    """
    Полная модель генерации подписей к видео, объединяющая:
    - Энкодер видео
    - Декодер с механизмом внимания
    """
    def __init__(self, vocab_size, feature_dim=2048, embed_dim=512, encoder_dim=512, 
                 decoder_dim=512, attention_dim=512, dropout=0.5):
        """
        Инициализация модели
        
        Аргументы:
            vocab_size (int): Размер словаря
            feature_dim (int): Размерность входных признаков
            embed_dim (int): Размерность эмбеддингов слов
            encoder_dim (int): Размерность скрытого слоя энкодера
            decoder_dim (int): Размерность скрытого слоя декодера
            attention_dim (int): Размерность механизма внимания
            dropout (float): Вероятность дропаута
        """
        super(VideoCaptioningModel, self).__init__()
        
        # Инициализация энкодера
        self.encoder = Encoder(
            feature_dim=feature_dim,
            hidden_dim=encoder_dim,
            dropout=dropout
        )
        
        # Инициализация декодера
        self.decoder = Decoder(
            vocab_size=vocab_size,
            embed_dim=embed_dim,
            encoder_dim=encoder_dim * 2,  # Учитываем двунаправленность энкодера
            hidden_dim=decoder_dim,
            attention_dim=attention_dim,
            dropout=dropout
        )
        
    def forward(self, features, captions, lengths):
        """
        Прямой проход через модель
        
        Аргументы:
            features (torch.Tensor): Признаки видео [batch_size, seq_len, feature_dim]
            captions (torch.Tensor): Истинные подписи [batch_size, max_caption_length]
            lengths (list): Фактические длины подписей
            
        Возвращает:
            torch.Tensor: Предсказания модели [batch_size, max_caption_length, vocab_size]
        """
        # Кодируем видео
        encoder_outputs, _ = self.encoder(features)
        
        # Декодируем подписи
        outputs = self.decoder(encoder_outputs, captions, lengths)
        
        return outputs

In [ ]:
# 4. Функции обучения модели

def train_epoch(model, dataloader, criterion, optimizer, epoch, device):
    """Обучение модели в течение одной эпохи
    
    Аргументы:
        model: Модель для обучения
        dataloader: Загрузчик обучающих данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        epoch: Номер текущей эпохи
        device: Устройство для вычислений (CPU/GPU)
        
    Возвращает:
        Среднее значение потерь за эпоху
    """
    model.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f'Эпоха {epoch} [Обучение]')
    
    for features, captions in progress_bar:
        # Подготовка данных
        features = features.to(device, non_blocking=True)
        captions = captions.to(device, non_blocking=True)
        lengths = torch.tensor([len(cap) for cap in captions], 
                             dtype=torch.long, device='cpu')
        
        # Обнуление градиентов
        optimizer.zero_grad(set_to_none=True)
        
        # Прямой проход
        outputs = model(features, captions, lengths)
        loss = criterion(outputs.view(-1, outputs.size(-1)), 
                        captions.view(-1))
        
        # Обратное распространение
        loss.backward()
        
        # Оптимизация
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        # Логирование
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    
    return total_loss / len(dataloader)


def validate(model, dataloader, criterion, device):
    """Валидация модели на отдельном наборе данных
    
    Аргументы:
        model: Модель для валидации
        dataloader: Загрузчик валидационных данных
        criterion: Функция потерь
        device: Устройство для вычислений
        
    Возвращает:
        Среднее значение потерь на валидации
    """
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        for features, captions, lengths in tqdm(dataloader, desc='Валидация'):
            # Подготовка данных
            features = features.to(device, non_blocking=True)
            captions = captions.to(device, non_blocking=True)

            # Прямой проход
            outputs = model(features, captions, lengths)
            loss = criterion(outputs.view(-1, outputs.size(-1)),
                           captions.view(-1))
            
            total_loss += loss.item()
    
    return total_loss / len(dataloader)


def train_with_mixed_precision(model, dataloader, criterion, optimizer, 
                             epoch, device, scaler, accumulation_steps=4):
    """Обучение с использованием смешанной точности
    
    Аргументы:
        model: Модель для обучения
        dataloader: Загрузчик данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        epoch: Номер эпохи
        device: Устройство для вычислений
        scaler: GradScaler для смешанной точности
        accumulation_steps: Шаги накопления градиентов
        
    Возвращает:
        Среднее значение потерь за эпоху
    """
    model.train()
    total_loss = 0.0
    optimizer.zero_grad(set_to_none=True)
    
    for i, (features, captions, lengths) in enumerate(tqdm(dataloader, 
                                                desc=f'Эпоха {epoch} [FP16]')):
        # Подготовка данных
        features = features.to(device, non_blocking=True)
        captions = captions.to(device, non_blocking=True)
        
        # Прямой проход в смешанной точности
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(features, captions, lengths)
            loss = criterion(outputs.view(-1, outputs.size(-1)),
                           captions.view(-1)) / accumulation_steps
        
        # Обратное распространение
        scaler.scale(loss).backward()
        
        # Обновление весов с накоплением
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(dataloader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
        
        total_loss += loss.item() * accumulation_steps
    
    return total_loss / len(dataloader)


def get_lr_scheduler(optimizer, warmup_epochs=5, total_epochs=100):
    """Создает планировщик learning rate с прогревом и косинусным затуханием
    
    Аргументы:
        optimizer: Оптимизатор
        warmup_epochs: Количество эпох прогрева
        total_epochs: Общее количество эпох
        
    Возвращает:
        Learning rate scheduler
    """
    def lr_lambda(epoch):
        # Линейный прогрев
        if epoch < warmup_epochs:
            return float(epoch + 1) / float(warmup_epochs)
        # Косинусное затухание
        progress = float(epoch - warmup_epochs) / float(max(1, total_epochs - warmup_epochs))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * progress)))
    
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [ ]:
# 5. Основной цикл обучения модели

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, 
               save_dir='checkpoints', save_every=5):
    """Основная функция для обучения модели
    
    Аргументы:
        model: Модель для обучения
        train_loader: DataLoader для обучающих данных
        val_loader: DataLoader для валидационных данных  
        criterion: Функция потерь
        optimizer: Оптимизатор
        num_epochs: Количество эпох обучения
        device: Устройство для вычислений (CPU/GPU)
        save_dir: Директория для сохранения чекпоинтов
        save_every: Частота сохранения чекпоинтов (в эпохах)
        
    Возвращает:
        Обученную модель и историю обучения
    """
    
    # Создаем директорию для сохранения чекпоинтов
    os.makedirs(save_dir, exist_ok=True)
    
    # Инициализируем историю обучения
    history = {
        'train_loss': [],  # Потери на обучении
        'val_loss': []     # Потери на валидации
    }
    
    # Инициализируем инструменты для оптимизации обучения
    scaler = torch.amp.GradScaler('cuda')  # Для смешанной точности (FP16/FP32)
    scheduler = get_lr_scheduler(optimizer, warmup_epochs=5, total_epochs=num_epochs)  # Планировщик learning rate
    
    # Цикл обучения по эпохам
    for epoch in range(1, num_epochs + 1):
        # Фаза обучения (со смешанной точностью)
        train_loss = train_with_mixed_precision(
            model=model,
            dataloader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            epoch=epoch,
            device=device,
            scaler=scaler
        )
        
        # Фаза валидации
        val_loss = validate(
            model=model,
            dataloader=val_loader,
            criterion=criterion,
            device=device
        )
        
        # Обновляем learning rate
        scheduler.step()
        
        # Сохраняем метрики
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
        # Выводим статистику
        print(f"Эпоха {epoch}/{num_epochs}, "
              f"Ошибка обучения: {train_loss:.4f}, "
              f"Ошибка валидации: {val_loss:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Сохраняем чекпоинт
        if epoch % save_every == 0:
            checkpoint_path = os.path.join(save_dir, f"checkpoint_epoch{epoch}.pt")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),  # Состояние модели
                'optimizer_state_dict': optimizer.state_dict(),  # Состояние оптимизатора
                'train_loss': train_loss,  # Потери на обучении
                'val_loss': val_loss,      # Потери на валидации
                'lr': optimizer.param_groups[0]['lr']  # Текущий learning rate
            }, checkpoint_path)
            print(f"Чекпоинт сохранен в {checkpoint_path}")
    
    # Сохраняем финальную модель
    final_path = os.path.join(save_dir, "final_model.pt")
    torch.save({
        'model_state_dict': model.state_dict(),  # Финальные веса модели
        'history': history  # Полная история обучения
    }, final_path)
    print(f"Финальная модель сохранена в {final_path}")
    
    return model, history

In [ ]:
# 6. Функции оценки качества модели

def predict_caption(model, video_features, vocab, max_length=20):
    """Генерация подписи к видео с помощью обученной модели
    
    Аргументы:
        model: Обученная модель генерации подписей
        video_features: Тензор признаков видео [1, seq_len, feature_dim]
        vocab: Словарь для преобразования слов
        max_length: Максимальная длина генерируемой подписи
        
    Возвращает:
        tuple: (подпись, веса внимания)
    """
    model.eval()
    device = next(model.parameters()).device
    
    with torch.no_grad():
        # Кодирование видео
        encoder_outputs, _ = model.encoder(video_features.to(device))
        
        # Инициализация декодера
        h, c = model.decoder.init_hidden_state(1)
        input_word = torch.tensor([vocab.word2idx['<start>']]).to(device)
        
        predicted_words = []
        attention_weights = []
        
        # Последовательная генерация
        for _ in range(max_length):
            # Шаг генерации
            embedding = model.decoder.embedding(input_word)
            context, alpha = model.decoder.attention(encoder_outputs, h)
            lstm_input = torch.cat([embedding, context], dim=1)
            h, c = model.decoder.lstm(lstm_input, (h, c))
            output = model.decoder.fc(h)
            
            # Получение следующего слова
            predicted_word_idx = output.argmax(1)
            attention_weights.append(alpha.cpu().numpy())
            
            # Проверка на конец последовательности
            if predicted_word_idx.item() == vocab.word2idx['<end>']:
                break
                
            predicted_words.append(vocab.idx2word[predicted_word_idx.item()])
            input_word = predicted_word_idx
    
    return ' '.join(predicted_words), attention_weights


def visualize_attention(video_frames, caption, attention_weights, grid_shape=(5, 8)):
    """Визуализация механизма внимания модели
    
    Аргументы:
        video_frames: Список кадров видео
        caption: Сгенерированная подпись
        attention_weights: Веса внимания
        grid_shape: Формат сетки для отображения
    """
    words = caption.split()
    if len(words) != len(attention_weights):
        print("Предупреждение: Несоответствие количества слов и весов внимания")
        return
    
    fig, axes = plt.subplots(len(words), 1, figsize=(12, 3*len(words)))
    
    for i, (word, weights) in enumerate(zip(words, attention_weights)):
        ax = axes[i] if len(words) > 1 else axes
        try:
            # Реорганизация весов внимания в сетку
            attention_grid = weights.reshape(grid_shape)
            im = ax.imshow(attention_grid, cmap='viridis')
            plt.colorbar(im, ax=ax)
            ax.set_title(f"Слово: '{word}'")
            ax.axis('off')
        except ValueError as e:
            print(f"Ошибка визуализации: {e}")
    
    plt.tight_layout()
    plt.show()


def evaluate_metrics(model, dataloader, vocab, device):
    """Комплексная оценка качества модели
    
    Аргументы:
        model: Обученная модель
        dataloader: Загрузчик данных для оценки
        vocab: Словарь
        device: Устройство вычислений
        
    Возвращает:
        dict: Словарь с метриками качества
    """
    model.eval()
    references = []
    hypotheses = []
    
    with torch.no_grad():
        for features, captions in tqdm(dataloader, desc='Оценка'):
            # Генерация подписи
            pred_caption, _ = predict_caption(model, features.to(device), vocab)
            hypotheses.append(pred_caption.split())
            
            # Обработка эталонной подписи
            ref = []
            for idx in captions[0].cpu().numpy():
                if idx == vocab.word2idx['<end>']:
                    break
                if idx not in [vocab.word2idx['<pad>'], vocab.word2idx['<start>']]:
                    ref.append(vocab.idx2word[idx])
            references.append([ref])
    
    # Вычисление метрик
    bleu4 = nltk.translate.bleu_score.corpus_bleu(
        references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))
    
    return {'bleu4': bleu4}


def find_optimal_batch_size(model, sample_data, criterion, max_batch_size=16):
    """Автоматический подбор оптимального размера батча
    
    Аргументы:
        model: Модель для тестирования
        sample_data: Пример данных (features, captions)
        criterion: Функция потерь для оценки результата
        max_batch_size: Максимальный размер батча для проверки
        
    Возвращает:
        int: Оптимальный размер батча
    """
    model.train()
    optimal_size = 1
    features, captions, lengths = sample_data
    
    for bs in range(1, max_batch_size + 1, 2):
        try:
            # Тестовый прогон
            batch_features = features.repeat(bs, 1, 1).to(device)
            batch_captions = captions.repeat(bs, 1).to(device)
            batch_lengths = lengths.repeat(bs) # Создаем batch_lengths

            # Передаем все три параметра в модель
            outputs = model(batch_features, batch_captions, batch_lengths)
            loss = criterion(outputs.view(-1, outputs.size(-1)), batch_captions.view(-1))
            loss.backward()
            
            optimal_size = bs
            print(f"Батч {bs} успешно обработан")
            
            # Очистка памяти
            del batch_features, batch_captions, batch_lengths, outputs, loss
            torch.cuda.empty_cache()
            
        except RuntimeError as e:
            if "памяти" in str(e).lower():
                print(f"Достигнут лимит памяти при батче {bs}")
                break
                
    print(f"Оптимальный размер батча: {optimal_size}")
    return optimal_size

def caption_collate_fn(batch):
    """Пользовательская функция для обработки батчей с подписями разной длины"""
    # Сортируем батч по длине подписей (по убыванию)
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    features, captions = zip(*batch)
    
    # Объединяем признаки
    features = torch.stack(features, 0)
    
    # Получаем длины подписей
    lengths = torch.LongTensor([len(cap) for cap in captions])
    
    # Определяем максимальную длину для паддинга
    max_length = max(lengths)
    
    # Создаем тензор с заполненными подписями
    padded_captions = torch.zeros(len(captions), max_length, dtype=torch.long)
    for i, cap in enumerate(captions):
        end = lengths[i]
        padded_captions[i, :end] = cap[:end]
    
    return features, padded_captions, lengths

In [ ]:
# 7. Визуализация и анализ результатов

def visualize_predictions(model, dataloader, vocab, num_samples=5):
    """
    Визуализация предсказаний модели для нескольких примеров
    
    Аргументы:
        model: Обученная модель
        dataloader: Загрузчик данных
        vocab: Словарь
        num_samples: Количество примеров для визуализации
    
    Процесс:
    1. Выбирает случайные примеры из датасета
    2. Генерирует подписи
    3. Сравнивает с эталонными подписями
    4. Визуализирует результаты
    """
    model.eval()  # Режим оценки
    
    samples = []
    count = 0
    
    with torch.no_grad():  # Без вычисления градиентов
        for features, captions, lengths in dataloader:
            if count >= num_samples:
                break
            
            # Переносим данные на устройство
            features = features.to(device)
            
            # Генерируем подпись
            predicted_caption, attention_weights = predict_caption(model, features, vocab)
            
            # Преобразуем эталонную подпись
            ground_truth = []
            for idx in captions[0].cpu().numpy():
                if idx == vocab.word2idx['<end>']:
                    break
                if idx not in [vocab.word2idx['<pad>'], vocab.word2idx['<start>']]:
                    ground_truth.append(vocab.idx2word[idx])
            
            ground_truth = ' '.join(ground_truth)
            
            # Сохраняем результаты
            samples.append({
                'features': features.cpu(),
                'predicted': predicted_caption,
                'ground_truth': ground_truth,
                'attention': attention_weights
            })
            
            count += 1
    
    # Создаем график для визуализации
    fig, axes = plt.subplots(num_samples, 1, figsize=(15, 5 * num_samples))
    
    for i, sample in enumerate(samples):
        ax = axes[i] if num_samples > 1 else axes
        
        # Отображаем предсказанную и эталонную подпись
        ax.text(0.5, 0.5, 
                f"Предсказание: {sample['predicted']}\nЭталон: {sample['ground_truth']}", 
                fontsize=12, ha='center')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

def analyze_attention_patterns(attention_weights, caption):
    """
    Анализ паттернов внимания для понимания работы модели
    
    Аргументы:
        attention_weights: Веса внимания
        caption: Сгенерированная подпись
    
    Анализирует:
    1. На какие кадры смотрит модель при генерации каждого слова
    2. Какие кадры наиболее важны для всей подписи
    3. Визуализирует распределение внимания
    """
    words = caption.split()
    
    # Создаем матрицу весов внимания
    attention_matrix = np.vstack([weights.reshape(-1) for weights in attention_weights])
    
    # Тепловая карта внимания
    plt.figure(figsize=(12, 8))
    plt.imshow(attention_matrix, aspect='auto', cmap='hot')
    plt.colorbar(label='Вес внимания')
    plt.xlabel('Позиция кадра')
    plt.ylabel('Позиция слова')
    plt.yticks(range(len(words)), words)
    plt.title('Распределение внимания')
    plt.show()
    
    # Анализ соответствия слов и кадров
    word_to_frame_map = attention_matrix.argmax(axis=1)
    
    print("Соответствие слов и кадров:")
    for i, word in enumerate(words):
        max_frame = word_to_frame_map[i]
        max_weight = attention_matrix[i, max_frame]
        print(f"Слово '{word}' максимально связано с кадром {max_frame} (вес {max_weight:.4f})")
    
    # Находим наиболее важные кадры
    frame_importance = attention_matrix.sum(axis=0)
    most_important_frames = frame_importance.argsort()[-5:][::-1]
    
    print("\nСамые важные кадры:")
    for frame_idx in most_important_frames:
        print(f"Кадр {frame_idx}: Общий вес внимания {frame_importance[frame_idx]:.4f}")

In [ ]:
# 8. Полный цикл обучения модели

def main():
    print("Подготовка данных...")
    # Пути к данным
    video_dir = "D:\\video_to_text\\YouTubeClips"  # Директория с видеофайлами
    caption_file = "D:\\video_to_text\\AllVideoDescriptions.txt"  # Файл с текстовыми описаниями
    feature_dir = "D:\\video_to_text\\features"  # Директория для сохранения предвычисленных признаков

    # Проверка наличия папки с признаками
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]
    features_exist = all(os.path.exists(os.path.join(feature_dir, os.path.splitext(f)[0] + ".npy")) for f in video_files)
    if not features_exist:
        print("Предварительный расчет признаков...")
        precompute_features(video_dir, feature_dir)
    else:
        print("Все признаки уже вычислены, пропускаем этап извлечения.")
       
    # 1. Предварительный расчет признаков из видео
    print("Предварительный расчет признаков...")
    precompute_features(video_dir, feature_dir)  # Извлечение и сохранение визуальных признаков
    
    # 2. Построение словаря из текстовых описаний
    print("Построение словаря...")
    vocab = build_vocab(caption_file, threshold=4)  # Создание словаря (порог вхождения слов = 4)
    vocab_size = len(vocab)  # Размер словаря
    print(f"Размер словаря: {vocab_size}")
    
    # 3. Создание и разделение датасета
    full_dataset = VideoDataset(feature_dir, caption_file, vocab)  # Инициализация набора данных
    train_size = int(0.7 * len(full_dataset))  # 70% данных для обучения
    val_size = int(0.15 * len(full_dataset))   # 15% для валидации
    test_size = len(full_dataset) - train_size - val_size  # Остаток для тестирования
    # Разделение данных с фиксированным seed для воспроизводимости
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42)
    )
    
    # 4. Подбор оптимального размера батча
    sample_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=caption_collate_fn)  # Тестовый загрузчик данных
    sample_data = next(iter(sample_loader))  # Получение одного примера данных
    # Инициализация модели для тестирования
    model = VideoCaptioningModel(vocab_size=vocab_size, feature_dim=2048, embed_dim=512, 
                                 encoder_dim=512, decoder_dim=512, attention_dim=512, dropout=0.5).to(device)
    criterion = nn.CrossEntropyLoss()  # Функция потерь
    # Автоподбор размера батча (макс = 16)
    batch_size = find_optimal_batch_size(model, sample_data, criterion, max_batch_size=16)
    
    # 5. Создание загрузчиков данных с оптимальным размером батча
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=caption_collate_fn)  # Для обучения
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=caption_collate_fn)     # Для валидации
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=caption_collate_fn)            # Для тестирования
    
    # 6. Инициализация компонентов обучения
    model = VideoCaptioningModel(vocab_size=vocab_size, feature_dim=2048, embed_dim=512, 
                                 encoder_dim=512, decoder_dim=512, attention_dim=512, dropout=0.5).to(device)
    criterion = nn.CrossEntropyLoss()  # Функция потерь для классификации
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)  # Оптимизатор с L2-регуляризацией
    
    # 7. Процесс обучения модели
    print("Обучение модели...")
    # Обучение в течение 20 эпох
    model, history = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)
    
    # 8. Визуализация процесса обучения
    plt.plot(history['train_loss'], label='Потери на обучении')  # Потери на обучении
    plt.plot(history['val_loss'], label='Потери на валидации')   # Потери на валидации
    plt.xlabel('Эпоха')  # Ось X - эпохи
    plt.ylabel('Потери') # Ось Y - значение потерь
    plt.legend()
    plt.title('Прогресс обучения')
    plt.grid(True)       # Включение сетки
    plt.show()           # Отображение графика
    
    # 9. Тестирование обученной модели
    print("Тестирование модели...")
    sample_features, sample_caption, sample_lengths = next(iter(test_loader))  # Получение тестового примера
    sample_features = sample_features.to(device)  # Перенос данных на устройство (GPU/CPU)
    caption, _ = predict_caption(model, sample_features, vocab)  # Генерация описания
    print(f"Сгенерированное описание: {caption}")  # Вывод результата

    # 10. Визуализация предсказаний для нескольких примеров
    print("Визуализация предсказаний...")
    visualize_predictions(model, test_loader, vocab, num_samples=3)  # Показать 3 примера
    
    # 11. Анализ паттернов внимания для сгенерированной подписи
    print("Анализ паттернов внимания...")
    analyze_attention_patterns(attention_weights, caption)  # Анализ весов внимания
    
    # Можно также извлечь кадры из видео для визуализации внимания с кадрами

    video_id = os.path.splitext(os.listdir(video_dir)[0])[0]
    video_path = os.path.join(video_dir, f"{video_id}.mp4")
    frames = extract_frames(video_path)
    visualize_attention(frames, caption, attention_weights)

In [ ]:
if __name__ == "__main__":
    main()